In [ ]:
# Configs
dataset_dir = r'C:\Users\fardin\Projects\EnhanceSEO\datasets\extractedURLs\url_classes.csv'

In [1]:
# from torchvision import datasets
# from torchtext.datasets import AG_NEWS
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize, TweetTokenizer
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
import time

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 47.9kB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 569kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:02<00:00, 109kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:04<00:00, 115kB/s]


In [3]:
from transformers import AutoModelForSequenceClassification
bert_model = AutoModelForSequenceClassification.from_pretrained(
  "bert-base-uncased",
  trust_remote_code=True
)

model.safetensors: 100%|██████████| 440M/440M [08:38<00:00, 850kB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# model="lxyuan/distilbert-base-multilingual-cased-sentiments-student"
# bert_tokenizer = BertTokenizer.from_pretrained(model)

In [5]:
# def get_text_and_labels(input_data):
#     labels = []
#     data = []
#     for label, d in input_data:
#         labels.append(label)
#         data.append(d)
#     labels = pd.DataFrame(labels, columns=['labels'], dtype=int)
#     data = pd.DataFrame(data, columns=['descriptions'], dtype=str)
#     return data, labels

In [6]:
df = pd.read_csv(r'data\CrawledWeb\TopicClassification\URLClassification\url_classes.csv')
# df.columns = ['Topic', 'Address']
df.dropna(inplace=True)
class_list = df.Topic.unique()
class_id = {t:i for i, t in enumerate(class_list)}
id_class = {i:t for i, t in enumerate(class_list)}
df_train, df_test = train_test_split(df, test_size=0.1, shuffle=True)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 1024
device

device(type='cuda')

In [8]:
# tokenizer = lambda s: list(urlparse(s))

In [9]:
# doc = "this is a sample sentence, do you understand this?"
# tokenizer = TweetTokenizer()
# lemmatizer = WordNetLemmatizer()
# for i in range(10000):
#     # tokens = word_tokenize(doc)
#     tokens = tokenizer.tokenize(doc)
#     [lemmatizer.lemmatize(t) for t in tokens]

In [10]:
# class VocabFactory():
    
#     def __init__(self) -> None:
#         self.tokens_dict = {}
#         self.lemmatizer = WordNetLemmatizer()
#         self.vocab_id = {}
#         self.id_vocab = {}
    
#     def create_vocab(self, documents, vocab_count=10000):
#         i = 0
#         for doc in documents:
#             if i %10000 == 0:
#                 print(i)
#             i+=1
#             tokens = word_tokenize(doc)
#             for token in tokens:
#                 token = self.lemmatizer.lemmatize(token)
#                 if token not in self.tokens_dict:
#                     self.tokens_dict[token] = 0
#                 self.tokens_dict[token] += 1
#         self.finalize(vocab_count)
    
#     def finalize(self, vocab_count=10000):
#         all_tokens = pd.DataFrame({'tokens': list(self.tokens_dict.keys()), 'counts': list(self.tokens_dict.values())})
#         all_tokens = all_tokens.sort_values(by=['counts'], ascending=False)
#         tokens = all_tokens['tokens'].values[:vocab_count]
#         self.vocab_id = {t: i for i, t in enumerate(tokens)}
#         self.id_vocab = {i: t for i, t in enumerate(tokens)}
        
        

In [11]:
# def yield_tokens(docs, tokenizer):
#     for doc in docs:
#         yield tokenizer(doc[0])

# vf = VocabFactory()
# vf.create_vocab(train_x, vocab_count=12000)
vocab_dict = {t: i for i, t in enumerate(bert_tokenizer.vocab)}#vf.vocab_id
vocab_size = len(bert_tokenizer.vocab)
# build_vocab_from_iterator(yield_tokens(train_x.values, tokenizer), specials=["<unk>"], max_tokens=10000)

In [12]:
class CustomDataset(Dataset):
    
    def __init__(self, X, y, num_classes, dictionary, tokenizer, doc_length=256) -> None:
        super().__init__()
        self.doc_length = doc_length
        y = torch.from_numpy(np.array([class_id[c] for c in y], dtype=np.longlong))
        self.y = torch.nn.functional.one_hot(y, num_classes=num_classes).float()
        self.dictionary = dictionary
        self.tokenizer = tokenizer
        self.vocab_size = len(self.dictionary)
        
        self.X = torch.zeros((len(X), doc_length), dtype=torch.int)
        for i, doc in enumerate(X):
            # tokens = self.tokenizer(doc)
            # print(doc)
            indices = torch.from_numpy(np.array(self.tokenizer(doc)['input_ids'], dtype=np.longlong))
            pad_size = max(self.doc_length - len(indices), 0)
            self.X[i] = torch.nn.functional.pad(indices[:self.doc_length], (0,pad_size))
        
        
    def __getitem__(self, index):
        # doc = self.X[index]
        # label = self.y[index]
        # tokens = self.tokenizer(doc)
        # tokens = [self.lemmatizer(t) for t in tokens]
        # indices = torch.from_numpy(np.array([self.dictionary[t] for t in tokens if t in self.dictionary], dtype=np.longlong))
        # pad_size = max(self.doc_length - len(indices), 0)
        # indices = torch.nn.functional.pad(indices[:self.doc_length], (0,pad_size))
        # token_vectors = torch.nn.functional.one_hot(indices, self.vocab_size)
        return self.X[index], self.y[index]
        
    def __len__(self):
        return len(self.y)
        

In [13]:
bert_tokenizer("My name is Fardin!")

{'input_ids': [101, 2026, 2171, 2003, 2521, 8718, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
lens = []
for a in df.Address:
    lens.append(len(a))
print(max(lens))
print(lens)

617
[37, 47, 66, 104, 85, 59, 90, 83, 90, 93, 65, 88, 51, 75, 118, 125, 86, 120, 53, 45, 20, 36, 23, 30, 23, 25, 22, 24, 25, 30, 31, 50, 36, 38, 25, 60, 74, 50, 68, 41, 24, 20, 22, 21, 20, 28, 20, 27, 33, 21, 50, 55, 39, 218, 43, 62, 35, 62, 42, 66, 61, 44, 46, 45, 62, 68, 71, 98, 77, 66, 24, 38, 51, 29, 30, 24, 26, 32, 40, 30, 90, 101, 24, 89, 45, 82, 38, 46, 69, 55, 37, 38, 45, 50, 44, 52, 56, 37, 39, 57, 46, 51, 24, 34, 42, 66, 68, 75, 62, 73, 47, 52, 55, 59, 110, 62, 56, 21, 59, 88, 24, 50, 32, 84, 44, 82, 42, 58, 40, 38, 22, 49, 52, 61, 113, 33, 74, 110, 27, 61, 46, 50, 42, 52, 41, 41, 118, 58, 74, 59, 43, 137, 43, 66, 43, 69, 170, 73, 59, 103, 41, 71, 50, 129, 55, 40, 102, 54, 28, 55, 40, 61, 34, 74, 95, 110, 95, 67, 89, 116, 61, 72, 66, 73, 79, 82, 70, 60, 86, 77, 66, 93, 78, 50, 45, 52, 94, 36, 25, 87, 45, 30, 118, 51, 67, 33, 28, 66, 56, 90, 42, 56, 49, 64, 63, 77, 77, 48, 69, 63, 38, 52, 40, 37, 32, 46, 66, 53, 81, 84, 52, 26, 30, 58, 77, 58, 97, 83, 99, 72, 43, 70, 46, 43, 2

In [15]:
tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()
train_dataset = CustomDataset(df_train.Address.values, df_train.Topic.values, len(class_id), vocab_dict, bert_tokenizer, doc_length=256)
test_dataset = CustomDataset(df_test.Address.values, df_test.Topic.values, len(class_id), vocab_dict, bert_tokenizer, doc_length=256)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, shuffle=False)

In [16]:
for val in df_train.Address.values[:1]:
    print(train_dataset.tokenizer(val).keys())
for val in df_train.Address.values[:2]:
    print(train_dataset.tokenizer(val)['input_ids'])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[101, 16770, 1024, 1013, 1013, 7479, 1012, 26866, 1012, 8917, 1013, 2739, 1013, 2865, 1011, 6325, 102]
[101, 16770, 1024, 1013, 1013, 12043, 15166, 29337, 5339, 14287, 1012, 6187, 1013, 25682, 1013, 5890, 1013, 2403, 1013, 3319, 1011, 8987, 1011, 1037, 1011, 8348, 1011, 5877, 25820, 1011, 2208, 1013, 102]


In [17]:
st = time.time()
X, y = next(iter(test_dataloader))
print(f'dataset time = {time.time() - st}')

dataset time = 0.6469950675964355


In [18]:
from copy import copy


class CNN_for_Text(nn.Module):
    
    def __init__(self, embedding_weight,batch_size, max_token_count, dropout=0.3, num_out_features=4, train_embedding=False, *args, **kwargs) -> None:
        super(CNN_for_Text, self).__init__(*args, **kwargs)
        self.batch_size = batch_size
        self.max_token_count = max_token_count
        
        self.embedding = torch.nn.Embedding(embedding_weight.shape[0], embedding_weight.shape[1])
        self.embedding.weight = torch.nn.Parameter(copy(embedding_weight))
        self.embedding.weight.requires_grad=train_embedding
        
        embedding_dim = embedding_weight.shape[1]
        self.conv1 = nn.Conv1d(embedding_dim, 64, kernel_size=5)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(64, 64, kernel_size=5)
        self.globalpool = nn.AdaptiveMaxPool1d(64)
        self.fc1 = nn.Linear(64 * max_token_count//4, 32)
        self.dropout = nn.Dropout(dropout)
        self.fc_out = nn.Linear(32, num_out_features)
        
    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.globalpool(x)
        x = F.relu(self.fc1(x.view(self.batch_size, -1)))
        x = self.dropout(x)
        x = self.fc_out(x)
        return x

In [19]:
model = CNN_for_Text(bert_model.bert.embeddings.word_embeddings.weight, batch_size=batch_size, max_token_count=256, num_out_features=len(class_id), train_embedding=True)

In [20]:
X.shape

torch.Size([1024, 256])

In [21]:
model(X).shape

torch.Size([1024, 12])

In [22]:
import numpy as np

In [23]:
import torchmetrics
import lightning as L
# from abc import abstractmethod

In [24]:

class ClassifierLightningModel(L.LightningModule):
    def __init__(
        self,
        model,
        num_classes,
        optimizer=None,
        loss_func=None,
        learning_rate=0.01,
        batch_size=64,
        lr_scheduler=None,
        user_lr_scheduler=False,
        min_lr=0.0,
    ):
        super(ClassifierLightningModel, self).__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model = model
        self.min_lr = min_lr
        # self.save_hyperparameters(ignore=["model"])
        self.save_hyperparameters("model", logger=False)
        self.optimizer = self._get_optimizer(optimizer)
        self.lr_scheduler = (
            self._get_lr_scheduler(lr_scheduler) if user_lr_scheduler else None
        )
        self.loss_func = loss_func
        self.train_losses = []
        self.val_losses = []
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    def on_train_epoch_start(self) -> None:
        param_groups = next(iter(self.optimizer.param_groups))
        if "lr" in param_groups and param_groups["lr"] is not None:
            current_learning_rate = float(param_groups["lr"])
            self.log(
                "lr",
                current_learning_rate,
                batch_size=self.batch_size,
                on_epoch=True,
                on_step=False,
            )

    def training_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.train()
        y_out = self(X)

        loss = self.loss_func(y_out.view(y.shape), y )
        self.train_losses.append(loss.detach().item())
        self.log(
            "train_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True,
            on_step=True,
        )
        
        self.train_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('train_acc', self.train_acc, prog_bar=True, on_epoch=True, on_step=True, batch_size=self.batch_size)
        
        return loss

    def validation_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.eval()
        y_out = self(X)
        loss = self.loss_func(y_out.view(y.shape), y )
        self.val_losses.append(loss.detach().item())

        self.log(
            "val_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True,
            on_step=True,
        )
        
        
        self.val_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('val_acc', self.val_acc, prog_bar=True, on_epoch=True, on_step=True, batch_size=self.batch_size)

    def configure_optimizers(self):
        if self.lr_scheduler is None:
            return self.optimizer

        return {
            "optimizer": self.optimizer,
            "lr_scheduler": {
                "scheduler": self.lr_scheduler,
                "monitor": "train_loss",
                "interval": "epoch",
                "frequency": 1,
            },
        }

    def update_learning_rate(self, learning_rate: float):
        self.learning_rate = learning_rate
        for g in self.optimizer.param_groups:
            g["lr"] = learning_rate

    def _get_optimizer(self, optimizer):
        return (
            optimizer
            if optimizer is not None
            else torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        )

    def _get_lr_scheduler(self, lr_scheduler):
        return (
            lr_scheduler
            if lr_scheduler is not None
            else torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, patience=5, factor=0.5, mode="min", min_lr=self.min_lr
            )
        )


In [25]:

lr=  0.001380384264602885
# 0.00010631317724117211
output_size = 128

In [26]:
len(bert_tokenizer.vocab)

30522

In [27]:
hidden_dim = 256
embedding_dim = 128
label_size = 1

classifier_torch_model = CNN_for_Text(bert_model.bert.embeddings.word_embeddings.weight, batch_size=batch_size, max_token_count=256, dropout=0.2, num_out_features=len(class_id), train_embedding=False).to(device)
optimizer = torch.optim.Adam(classifier_torch_model.parameters(), lr=lr, weight_decay=0.00012)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90, 120],gamma=0.5, verbose=True)
loss_func = torch.nn.BCEWithLogitsLoss()
classfier_lightning_model = ClassifierLightningModel(classifier_torch_model, 
                                                     num_classes=len(class_id),
                                            learning_rate=lr,
                                            batch_size=batch_size,
                                            optimizer=optimizer,
                                            loss_func=loss_func,
                                            lr_scheduler=lr_scheduler,
                                            user_lr_scheduler=True
                                            ).to(device)

Adjusting learning rate of group 0 to 1.3804e-03.


c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [28]:
# from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger
import lightning as L

# from scripts.utils.CustomCallbacks.ModelCheckpoint import CustomModelCheckpoint

# callbacks = [
#         CustomModelCheckpoint(dirpath=r'models\model2_word_embedding-256-2', filename='str_embedding', every_n_epochs=1, mode='min', monitor='train_loss', save_on_train_epoch_end=True),
#         ModelCheckpoint(save_top_k=5, mode='min', monitor='train_loss', save_last=True)
#         ]
trainer = L.Trainer(
            # callbacks=callbacks,
            max_epochs=400,
            accelerator= 'gpu' if device==torch.device("cuda") else 'cpu',
            logger=CSVLogger(save_dir='logs/', name='log2'), 
            num_sanity_val_steps=0,
        #     default_root_dir='models\model2_word_embedding-256-2'
        )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
# del bert_model

In [30]:
# from lightning.pytorch.tuner import Tuner
# tuner = Tuner(trainer)
# tuning_result = tuner.lr_find(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader, min_lr=0.00001,max_lr=0.01, num_training=100)

# fig = tuning_result.plot(suggest=True)
# fig.show()

In [31]:
# max_epochs = 1000
# trainer.fit_loop.max_epochs = max_epochs
trainer.fit(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)


You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | CNN_for_Text       | 23.8 M
1 | loss_func | BCEWithLogitsLoss  | 0     
2 | train_acc | MulticlassAccuracy | 0     
3 | val_acc   | MulticlassAccuracy | 0     
4 | test_acc  | MulticlassAccuracy | 0     
-------------------------------------------------
397 K     Trainable params
23.4 M    Non-trainable params
23.8 M    Total params
95.355    Total estimated model params size (MB)
c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer

Epoch 399: 100%|██████████| 31/31 [00:03<00:00,  9.97it/s, v_num=494, train_loss_step=0.0679, train_acc_step=0.889, val_loss_step=0.112, val_acc_step=0.767, val_loss_epoch=0.109, val_acc_epoch=0.768, train_loss_epoch=0.0693, train_acc_epoch=0.879]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 399: 100%|██████████| 31/31 [00:03<00:00,  8.14it/s, v_num=494, train_loss_step=0.0679, train_acc_step=0.889, val_loss_step=0.112, val_acc_step=0.767, val_loss_epoch=0.109, val_acc_epoch=0.768, train_loss_epoch=0.0693, train_acc_epoch=0.879]


In [32]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from torchmetrics import ConfusionMatrix

def calculate_metrics(cl_model):
    cm = ConfusionMatrix(task="multiclass", num_classes=len(class_id))

    y_pred = []
    y_true = []

    cl_model = cl_model.eval()
    cl_model.to(device)
    for X, y in tqdm(test_dataloader):
        X = X.to(device)
        with torch.no_grad():
            y_p = cl_model(X)
            y_p = y_p.cpu()
        y_pred.append(y_p)
        y_true.append(y)
    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)
    y_pred2 = torch.argmax(y_pred, dim=1)
    y_true2 = torch.argmax(y_true, dim=1)
    print(f'classification report: \n {classification_report(y_true2, y_pred2, digits=4)}')
    print(f'confusion matrix:\n {cm(y_pred2, y_true2)}')
    print('================================')


In [33]:
classfier_lightning_model.model = classfier_lightning_model.model.eval()
classfier_lightning_model = classfier_lightning_model.eval()
calculate_metrics(classfier_lightning_model)

100%|██████████| 3/3 [00:00<00:00, 15.08it/s]


classification report: 
               precision    recall  f1-score   support

           0     0.8065    0.7380    0.7707       271
           1     0.8269    0.8417    0.8342       278
           2     0.9127    0.8462    0.8782       247
           3     0.8327    0.7770    0.8038       269
           4     0.6900    0.6529    0.6709       242
           5     0.6920    0.6458    0.6681       240
           6     0.7545    0.7770    0.7656       269
           7     0.8588    0.8295    0.8439       264
           8     0.7148    0.7355    0.7250       276
           9     0.6768    0.7882    0.7283       255
          10     0.6697    0.7690    0.7159       290
          11     0.8519    0.8070    0.8288       171

    accuracy                         0.7676      3072
   macro avg     0.7739    0.7673    0.7695      3072
weighted avg     0.7716    0.7676    0.7684      3072

confusion matrix:
 tensor([[200,   3,   3,   0,   1,   3,  11,   3,  37,   2,   3,   5],
        [  6, 234, 